# Retrieval Augmented Generation (RAG)
Building the Journal page navigator.

In [ ]:
%pip install langchain
%pip install python-dotenv

In [1]:
## Initialize OpenAI API
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']
print('ready!')

ready!


## Document Loading


In [3]:
## Document Loading
from langchain.document_loaders import WebBaseLoader
import requests
from bs4 import BeautifulSoup

# Define the base URL
base_url = "https://antfriend.github.io/journals"

# Function to get all HTML document URLs from the base URL
def get_document_urls(base_url):
    page = requests.get(base_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    links = soup.find_all('a')

    # Check if href is not None and ends with '.html'
    # An anchor tag with an href of None is a link to the current page
    document_urls = [link.get('href') for link in links if link.get('href') and link.get('href').endswith('.html')]
    document_urls = [base_url + '/' + url for url in document_urls]
    return document_urls

# Get all document URLs
document_urls = get_document_urls(base_url)
print(len(document_urls), 'journal pages from:')
print(document_urls[0])
print('to:')
print(document_urls[-1])

# Load and process each document
documents = []
for url in document_urls:
    # Initialize the WebBaseLoader
    loader = WebBaseLoader(url)
    document = loader.load()
    documents.append(document)

# `documents` now contains the loaded HTML content from each URL
print(len(documents), 'documents loaded')


13 journal pages from:
https://antfriend.github.io/journals/waves_15.html
to:
https://antfriend.github.io/journals/vectors_32.html
13 documents loaded


## Splitting

In [ ]:
## Splitting
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

## Storage


In [ ]:
## Storage

## Retrieval

In [ ]:
## Retrieval
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

## Output

In [ ]:
## Output